In [7]:
#Turns a .h5 model file using custom metric into a TFLite flat buffer 


#see
    #https://www.tensorflow.org/lite/convert/python_api
    #https://www.tensorflow.org/lite/performance/post_training_quantization
    #https://www.tensorflow.org/lite/microcontrollers/build_convert#operation-support
    #https://www.tensorflow.org/versions/r1.14/api_docs/python/tf/lite/TFLiteConverter
    #https://github.com/keras-team/keras/issues/10104

#imports

import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow.lite
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import numpy as np


In [8]:
# custom metric: auc

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc



In [24]:

model_name = "/Users/laurenogden/gunshot_detection/raspberry_pi/models/gunshot_sound_model.h5"
converted_name = "/Users/laurenogden/gunshot_detection/raspberry_pi/models/converted_gunshot_sound_model.tflite"

custom_objects={"auc": auc}


interpreter = tf.lite.Interpreter(model_path=converted_name)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(type(input_details[0]['shape']))

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
print(type(input_shape))
print(type(input_data))


interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
print(output_details[0])
print(output_details[0]['index'])





[{'name': 'input_1', 'index': 46, 'shape': array([    1, 44100,     1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[1.0000000e+00 8.4182965e-26]]
{'name': 'dense_2/Softmax', 'index': 42, 'shape': array([1, 2], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
42


In [ ]:
model = load_model(model_name, custom_objects)


converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name, custom_objects)
print("\n made the converter using from_keras_model \n")

#post-training quantization
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
#print(" did post training quantization \n")

#convert
tflite_model = converter.convert()
print(" converted successfully \n")

#save
open("/Users/laurenogden/gunshot_detection/raspberry_pi/models/converted_gunshot_sound_model.tflite", "wb").write(tflite_model)
print(" saved successfully \n")